In [9]:
# 1. Write a decorator that ensures a function is only called by users with a specific role. 
# Each function should have an user_type with a string type in kwargs. Example:
# @is_admin
# def show_customer_receipt(user_type: str):
#     # Some very dangerous operation

# show_customer_receipt(user_type='user')
# > ValueError: Permission denied

# show_customer_receipt(user_type='admin')
# > function pass as it should be

def is_admin(func):
    def wrapper(**kwargs):
        if "admin" in kwargs.values():
            return func(**kwargs)
        else:
            raise ValueError("Acess denied") 
    return wrapper

@is_admin
def show_customer_receipt(user_type: str):
    print("Here is your receipt")

show_customer_receipt(user_type='admin')
show_customer_receipt(user_type="user")

    

{'user_type': 'admin'}
Here is your receipt
{'user_type': 'user'}


ValueError: Acess denied

In [77]:
# 2. Write a decorator that wraps a function in a try-except block and prints an error if any type of error has happened. Example:
# @catch_errors
# def some_function_with_risky_operation(data):
#     print(data['key'])

# some_function_with_risky_operation({'foo': 'bar'})
# > Found 1 error during execution of your function: KeyError no such key as foo

# some_function_with_risky_operation({'key': 'bar'})
# > bar

# def catch_errors(func):
#     def wrapper(*args):
#         if args[-1] != 0:
#             return func(*args)
#         else:
#             raise ZeroDivisionError("There is attempt to divide by zero")
#     return wrapper

# @catch_errors
# def division_with_zero(number:int, number_2:int) -> int:
#     print(number/number_2)

# division_with_zero(0, 2)
# division_with_zero(2, 0)

# Another case.

# def catch_errors_2(func):
#     def wrapper(*args):
#         for arg in args:
#             if type(arg) == dict:
#                 for key in arg.keys():
#                     try:
#                          func(*args)
#                     except KeyError:
#                         print(f"Found 1 error during execution of your function: KeyError no such key as {key}")
#             else:
#                 raise TypeError("This is not a dict")                
#     return wrapper

# Simplified soution.

def catch_errors_2(func):
    def wrapper(some_data):
        if type(some_data) == dict:
            for key in some_data.keys():
                try:
                    func(some_data)
                except KeyError:
                    print(f"Found 1 error during execution of your function: KeyError no such key as {key}")
        else:
            raise TypeError(f"This is not a dict. It's a {type(some_data).__name__}")                
    return wrapper

@catch_errors_2
def some_function_with_risky_operation(data):
    print(data['key'])

some_function_with_risky_operation({'key': 'bar'})
some_function_with_risky_operation({'foo': 'bar'})
some_function_with_risky_operation(('key', 'bar'))

bar
Found 1 error during execution of your function: KeyError no such key as foo


TypeError: This is not a dict. It's a tuple

In [31]:
# 3. Optional: Create a decorator that will check types. It should take a function with arguments and validate inputs with annotations. 
# It should work for all possible functions. Don`t forget to check the return type as well

# @check_types
# def add(a: int, b: int) -> int:
#     return a + b

# add(1, 2)
# > 3
# add("1", "2")
# > TypeError: Argument a must be int, not str


def check_types(func):
    def wrapper(*args):
        annons = func.__annotations__
        #print(annons)

        for i, arg in enumerate(args):
            arg_name = list(annons.keys())[i]
            arg_type = annons.get(arg_name)
            if arg_type and not isinstance(arg, arg_type):
                raise TypeError(f"Argument '{arg_name}' must be of type {arg_type.__name__}, not {type(arg).__name__}")
        
        result = func(*args)
        
        return_type = annons.get('return')
        if return_type and not isinstance(result, return_type):
            raise TypeError(f"Function should return {return_type.__name__}, not {result.__name__}.")
        
        return result
    
    return wrapper    


# Extended.

def full_check_types(func):
    def wrapper(*args, **kwargs):
        annons = func.__annotations__
        #print(annons)

        for i, arg in enumerate(args):
            arg_name = list(annons.keys())[i]
            arg_type = annons.get(arg_name)
            if arg_type and not isinstance(arg, arg_type):
                raise TypeError(f"Argument '{arg_name}' must be of type {arg_type.__name__}, not {type(arg).__name__}.")
        
        for arg_name, arg_value in kwargs.items():
            arg_type = annons.get(arg_name)
            if arg_type and not isinstance(arg_value, arg_type):
                raise TypeError(f"Argument '{arg_name}' should be of type {arg_type.__name__}, not {type(arg_value).__name__}.")
        
        result = func(*args, **kwargs)
        
        return_type = annons.get('return')
        if return_type and not isinstance(result, return_type):
            raise TypeError(f"Function should return {return_type.__name__}, not {result.__name__}.")
        
        return result
    
    return wrapper    


# @check_types
# def add(a : int, b : int) -> int:
#     return a + b

# print(add(1, 2))
# print(add(1, "2"))




@full_check_types
def add(a : int, b : int) -> int:
    return a + b

print(add(a=1, b=2))
print(add(a=1, b="2"))

3


TypeError: Argument 'b' should be of type int, not str.

In [35]:
# 4. Optional: Create a function that caches the result of a function, 
# so that if it is called with the same argument multiple times, it returns the cached result first instead of re-executing the function.

import time
from functools import wraps, lru_cache


def cache(user_func):
    cache = {}
    def wrapper(*args):
        if args in cache.keys():
            return cache[args]
        else:
            result = user_func(*args)
            cache[args] = result
            return result
    return wrapper

def repeat(number_of_times):
    def decorate(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            for _ in range(number_of_times):
                func(*args, **kwargs)
        return wrapper
    return decorate

def timeit(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.perf_counter()
        result = func(*args, **kwargs)
        end = time.perf_counter()
        print(f'{func.__name__} took {end - start:.6f} seconds to complete')
        return result
    return wrapper


@timeit
@repeat(25000000)
def add(a: int, b: int) -> int:
    return (((a + b)*(b+a))**3)/(a+b)**9

add(1, 23)

@timeit
@repeat(25000000)
@cache
def add(a: int, b: int) -> int:
    return (((a + b)*(b+a))**3)/(a+b)**9

add(1, 23)

# compare with lib
@timeit
@repeat(25000000)
@lru_cache
def add(a: int, b: int) -> int:
    return (((a + b)*(b+a))**3)/(a+b)**9
add(1, 23)

# Guess it's work)

add took 7.457482 seconds to complete
wrapper took 5.740481 seconds to complete
add took 1.750110 seconds to complete


In [7]:
5. # Optional: Write a decorator that adds a rate-limiter to a function, so that it can only be called a certain amount of times per minute

import time

def rate_limits(max_calls, period):
    def decorator(func):
        calls = 0
        last_reset = time.time()

        def wrapper(*args, **kwargs):
            nonlocal calls, last_reset

            elapsed = time.time() - last_reset

            if elapsed > period:
                calls = 0
                last_reset = time.time()

            if calls >= max_calls:
                raise Exception("Rate limit exceeded. Please try again later.")

            calls += 1

            return func(*args, **kwargs)

        return wrapper
    return decorator


@repeat(6)
@rate_limits(5, 60)  # 2 per second at most
def PrintNumber(num):
    print (num)

PrintNumber(9)

9
9
9
9
9


Exception: Rate limit exceeded. Please try again later.